# Trend Analysis on Twitter live data using Spark Streaming

Import sparkContext & StreamingContext from PySpark library.

In [2]:
from __future__ import print_function
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

Create a sparkContext with AppName "SteramingTwitterAnalysis".<br>
Setting the LogLevel of SparkContext to ERROR. This will not print all the logs which are INFO or WARN level.<br>
Create Spark Streaming Context using SC (spark context). parameter 10 is the batch interval. <br>
Every 10 second the analysis will be done.

In [3]:
sc1 = SparkContext(appName="SteramingTwitterAnalysis")
sc1.setLogLevel("ERROR")
ssc1 = StreamingContext(sc1, 10)

Connect to socket broker using ssc (spark streaming context)<br>
Host : "172.31.20.58" (localhost) & port : 5555

In [4]:
socket_stream = ssc1.socketTextStream("127.0.0.1", 9912)

window function parameter sets the Window length. All the analysis will be done on tweets stored for 60 secs.

In [5]:
lines = socket_stream.window( 20 )

### Process the Stream:
1. Receives tweet message, stored in lines. **Input DStream**
2. splits the messages into words. **Apply transformation on DStream : flatMap**
3. filters all the words which start with a hashtag(#). **transformation : filter**
4. converts the words to lowercase. **transformation : map**
5. maps each tag to (word, 1). **transformation : map**
6. then reduces and counts occurrences of each hash tag. (action : reduceByKey) hashtags = **output DStream**

In [6]:
hashtags = lines.flatMap( lambda text: text.split( " " ) ).filter( lambda word: word.lower().startswith("#") ).map( lambda word: ( word.lower()
, 1 ) ).reduceByKey( lambda a,b:a+b)

Sort the hashtags based on the counts in decreasing order

In [7]:
author_counts_sorted_dstream = hashtags.transform(lambda foo:foo.sortBy(lambda x:x[0].lower()).sortBy(lambda x:x[1],ascending=False))

Print the final analysis: Most popular hashtags on streaming twitter data

In [8]:
author_counts_sorted_dstream.pprint()

### Starting the Spark Streaming:
Spark Streaming code we have written till now will not execute, untill we start the ssc.<br>
ssc.start() will start the spark streaming context. This is the Action for the whole code. <br>
Now it'll create the lineage & DAG & do the lazy evaluation & start running the whole sequesnce of code.


In [9]:
ssc.start()

awaitTermination() is very important to stop the SSC.<br> 
When we kill this python process then this signal will be sent to awaitTermination() function.<br> 
it will finally stop the spark streaming job.

In [10]:
ssc.awaitTermination()

Py4JJavaError: An error occurred while calling o22.awaitTermination.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/pyspark/streaming/util.py", line 65, in call
    r = self.func(t, *rdds)
  File "/anaconda3/lib/python3.6/site-packages/pyspark/streaming/dstream.py", line 631, in <lambda>
    self.func = lambda t, rdd: func(t, prev_func(t, rdd))
  File "/anaconda3/lib/python3.6/site-packages/pyspark/streaming/dstream.py", line 295, in <lambda>
    func = lambda t, rdd: oldfunc(rdd)
  File "<ipython-input-7-1b3177cf4a8c>", line 1, in <lambda>
    author_counts_sorted_dstream = hashtags.transform(lambda foo:foo.sortBy(lambda x:x[0].lower()).sortBy(lambda x:x[1],ascending=False))
  File "/anaconda3/lib/python3.6/site-packages/pyspark/rdd.py", line 717, in sortBy
    return self.keyBy(keyfunc).sortByKey(ascending, numPartitions).values()
  File "/anaconda3/lib/python3.6/site-packages/pys

In [11]:
import settings
import tweepy
import dataset
from textblob import TextBlob
from sqlalchemy.exc import ProgrammingError
import json


ModuleNotFoundError: No module named 'settings'